In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown

In [ ]:
#Helper Fuction
def import_from_gdrive(id_file, file_name):
  link = 'https://drive.google.com/uc?id=' + id_file
  output_file = '/content/' + file_name
  gdown.download(link, output_file, quiet=False)
  df = pd.read_csv(file_name)
  return df

In [ ]:
link_train_combined = '1RrJ73EqxTIME-tlSAU6Ob8n5_pxRbH_V'
link_test_combined = '1HkAMyEvIKKMXKuLPXMdbV_78vM00KXX0'
train = import_from_gdrive(link_train_combined,'train.csv')
test = import_from_gdrive(link_test_combined,'test.csv')

Downloading...
From: https://drive.google.com/uc?id=1RrJ73EqxTIME-tlSAU6Ob8n5_pxRbH_V
To: /content/train.csv
100%|██████████| 78.3M/78.3M [00:00<00:00, 110MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HkAMyEvIKKMXKuLPXMdbV_78vM00KXX0
To: /content/test.csv
100%|██████████| 25.0M/25.0M [00:00<00:00, 176MB/s]


#Data Preparation

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Columns to be encoded
columns_to_encode = ['weekend', 'libur', 'aktivitas', 'cycleway', 'highway', 'sidewalk', 'surface']

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform on training data
encoded_train = encoder.fit_transform(train[columns_to_encode])
encoded_train_df = pd.DataFrame(encoded_train.toarray(), columns=encoder.get_feature_names_out(columns_to_encode))

# Transform test data (do not fit again)
encoded_test = encoder.transform(test[columns_to_encode])
encoded_test_df = pd.DataFrame(encoded_test.toarray(), columns=encoder.get_feature_names_out(columns_to_encode))

# Concatenate encoding results with original data
train_clean = pd.concat([train, encoded_train_df], axis=1)
test_clean = pd.concat([test, encoded_test_df], axis=1)


In [ ]:
X_train = train_clean.drop(['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface'], axis= 1)
X_test = test_clean.drop(['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface'], axis= 1)
y_train = train_clean['rerata_kecepatan']

In [ ]:
ID = X_test['id']
X_test = X_test.drop(['id','kelompok'], axis=1)

In [ ]:
X_train = X_train.drop(['rerata_kecepatan','kelompok'], axis=1)

In [ ]:
X_train["maxspeed_mph"] = X_train["maxspeed_mph"] * 1.6
X_train.rename(columns={"maxspeed_mph": "maxspeed_kph"}, inplace=True)

X_test["maxspeed_mph"] = X_test["maxspeed_mph"] * 1.6
X_test.rename(columns={"maxspeed_mph": "maxspeed_kph"}, inplace=True)

#Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(318918, 32)
(127489, 32)
(79730, 32)


#LBGM

In [ ]:
import lightgbm as lgb

In [ ]:
X_train = X_train.drop(['titik1','titik2'], axis= 1)
X_test = X_test.drop(['titik1','titik2'], axis= 1)
X_val = X_val.drop(['titik1','titik2'], axis= 1)

In [ ]:
params = {
    "application": "regression",
    "boosting": "gbdt",
    "metric": "rmse",
    "num_leaves": 70,
    "max_depth": 9,
    "learning_rate": 0.01,
    "bagging_fraction": 0.85,
    "feature_fraction": 0.8,
    "min_split_gain": 0.02,
    "min_child_samples": 150,
    "min_child_weight": 0.02,
    "lambda_l2": 0.0475,
    "verbosity": -1,
    "data_random_seed": 17,
}

# Additional parameters:
early_stop = 500
verbose_eval = 100
num_rounds = 10000

d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_val, label=y_val)
valid_sets = [d_train, d_valid]

lgbm_model = lgb.train(params, train_set=d_train,num_boost_round=num_rounds, valid_sets=valid_sets, verbose_eval=verbose_eval,early_stopping_rounds=early_stop,)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 500 rounds
[100]	training's rmse: 5.86235	valid_1's rmse: 5.88905
[200]	training's rmse: 5.02771	valid_1's rmse: 5.0625
[300]	training's rmse: 4.64694	valid_1's rmse: 4.68426
[400]	training's rmse: 4.39689	valid_1's rmse: 4.43668
[500]	training's rmse: 4.23157	valid_1's rmse: 4.27161
[600]	training's rmse: 4.11458	valid_1's rmse: 4.15506
[700]	training's rmse: 4.01762	valid_1's rmse: 4.05856
[800]	training's rmse: 3.95071	valid_1's rmse: 3.99207
[900]	training's rmse: 3.89266	valid_1's rmse: 3.93459
[1000]	training's rmse: 3.82987	valid_1's rmse: 3.87282
[1100]	training's rmse: 3.78079	valid_1's rmse: 3.82512
[1200]	training's rmse: 3.73151	valid_1's rmse: 3.77704
[1300]	training's rmse: 3.68505	valid_1's rmse: 3.73191
[1400]	training's rmse: 3.64722	valid_1's rmse: 3.69561
[1500]	training's rmse: 3.61138	valid_1's rmse: 3.66109
[1600]	training's rmse: 3.57799	valid_1's rmse: 3.62907
[1700]	training's rmse: 3.55023	valid_1's rmse: 3.60

In [ ]:
# Prediksi dengan model yang telah dilatih
y_pred = lgbm_model.predict(X_test, num_iteration=lgbm_model.best_iteration)
y_pred

array([40.8949569 , 40.95647212, 36.67918139, ..., 41.46447038,
       35.76795108, 46.63846556])

In [ ]:
output = pd.DataFrame({'id': ID,
                       'rerata_kecepatan': y_pred})

In [ ]:
output.describe()

,id,rerata_kecepatan
count,127489.000000,127489.000000
mean,63744.000000,35.425765
std,36803.048572,7.622874
min,0.000000,5.282685
25%,31872.000000,31.352633
50%,63744.000000,36.150891
75%,95616.000000,40.625222
max,127488.000000,53.972803


##Tunning

In [ ]:
pip install optuna

In [ ]:
import lightgbm as lgb
import optuna

def objective(trial):
    params = {
        "application": "regression",
        "boosting": "gbdt",
        "metric": "rmse",
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.5),
        "min_child_samples": trial.suggest_int("min_child_samples", 50, 200),
        "min_child_weight": trial.suggest_float("min_child_weight", 0.001, 0.1),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 0.1),
        "verbosity": -1,
        "data_random_seed": 17,
    }

    d_train = lgb.Dataset(X_train, label=y_train)
    d_valid = lgb.Dataset(X_val, label=y_val)
    valid_sets = [d_train, d_valid]

    lgbm_model = lgb.train(
        params,
        train_set=d_train,
        num_boost_round=num_rounds,
        valid_sets=valid_sets,
        verbose_eval=verbose_eval,
        early_stopping_rounds=early_stop,
    )

    y_pred = lgbm_model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2023-08-10 04:13:01,557] A new study created in memory with name: no-name-d65564a3-7faa-4150-86c1-494d3b241e0a
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 500 rounds
[100]	training's rmse: 7.58183	valid_1's rmse: 7.5943
[200]	training's rmse: 7.09855	valid_1's rmse: 7.11373
[300]	training's rmse: 6.69263	valid_1's rmse: 6.70988
[400]	training's rmse: 6.35173	valid_1's rmse: 6.37144
[500]	training's rmse: 6.06397	valid_1's rmse: 6.08607
[600]	training's rmse: 5.81537	valid_1's rmse: 5.83935
[700]	training's rmse: 5.60808	valid_1's rmse: 5.63424
[800]	training's rmse: 5.42423	valid_1's rmse: 5.45204
[900]	training's rmse: 5.26838	valid_1's rmse: 5.29795
[1000]	training's rmse: 5.13519	valid_1's rmse: 5.16641
[1100]	training's rmse: 5.02399	valid_1's rmse: 5.05651
[1200]	training's rmse: 4.92162	valid_1's rmse: 4.95551
[1300]	training's rmse: 4.82655	valid_1's rmse: 4.86164
[1400]	training's rmse: 4.74286	valid_1's rmse: 4.77909
[1500]	training's rmse: 4.66669	valid_1's rmse: 4.70406
[1600]	training's rmse: 4.60085	valid_1's rmse: 4.63929
[1700]	training's rmse: 4.54207	valid_1's rmse: 4.58

[I 2023-08-10 04:21:54,310] Trial 0 finished with value: 3.41871882109547 and parameters: {'num_leaves': 117, 'max_depth': 11, 'learning_rate': 0.0015181599593361357, 'bagging_fraction': 0.7587158539700535, 'feature_fraction': 0.5686430126815472, 'min_split_gain': 0.2457093740281075, 'min_child_samples': 75, 'min_child_weight': 0.04515300232751004, 'lambda_l2': 0.09169378908024897}. Best is trial 0 with value: 3.41871882109547.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via '

Training until validation scores don't improve for 500 rounds
[100]	training's rmse: 7.05337	valid_1's rmse: 7.06812
[200]	training's rmse: 6.29121	valid_1's rmse: 6.30977
[300]	training's rmse: 5.75062	valid_1's rmse: 5.77378
[400]	training's rmse: 5.36576	valid_1's rmse: 5.39258
[500]	training's rmse: 5.0839	valid_1's rmse: 5.11347
[600]	training's rmse: 4.87336	valid_1's rmse: 4.90527
[700]	training's rmse: 4.71338	valid_1's rmse: 4.74702
[800]	training's rmse: 4.58001	valid_1's rmse: 4.61517
[900]	training's rmse: 4.47489	valid_1's rmse: 4.51223
[1000]	training's rmse: 4.37954	valid_1's rmse: 4.41801
[1100]	training's rmse: 4.28698	valid_1's rmse: 4.3258
[1200]	training's rmse: 4.20563	valid_1's rmse: 4.24544
[1300]	training's rmse: 4.12791	valid_1's rmse: 4.16877
[1400]	training's rmse: 4.05277	valid_1's rmse: 4.0936
[1500]	training's rmse: 3.9889	valid_1's rmse: 4.02991
[1600]	training's rmse: 3.93003	valid_1's rmse: 3.97157
[1700]	training's rmse: 3.87904	valid_1's rmse: 3.92119

[I 2023-08-10 04:28:58,962] Trial 1 finished with value: 3.2148305973362876 and parameters: {'num_leaves': 105, 'max_depth': 15, 'learning_rate': 0.0028473405763764703, 'bagging_fraction': 0.989551972008968, 'feature_fraction': 0.927053751501755, 'min_split_gain': 0.3483247463318934, 'min_child_samples': 169, 'min_child_weight': 0.06986747041330515, 'lambda_l2': 0.04708248294896207}. Best is trial 1 with value: 3.2148305973362876.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback vi

...

Training until validation scores don't improve for 500 rounds
[100]	training's rmse: 4.10928	valid_1's rmse: 4.15789
[200]	training's rmse: 3.65282	valid_1's rmse: 3.70893
[300]	training's rmse: 3.45808	valid_1's rmse: 3.52164
[400]	training's rmse: 3.33739	valid_1's rmse: 3.40951
[500]	training's rmse: 3.25618	valid_1's rmse: 3.33553
[600]	training's rmse: 3.18888	valid_1's rmse: 3.27722
[700]	training's rmse: 3.14446	valid_1's rmse: 3.24121
[800]	training's rmse: 3.11155	valid_1's rmse: 3.2155
[900]	training's rmse: 3.0776	valid_1's rmse: 3.1889
[1000]	training's rmse: 3.04616	valid_1's rmse: 3.16532
[1100]	training's rmse: 3.02625	valid_1's rmse: 3.15131
[1200]	training's rmse: 3.00468	valid_1's rmse: 3.13729
[1300]	training's rmse: 2.9857	valid_1's rmse: 3.12477
[1400]	training's rmse: 2.9665	valid_1's rmse: 3.11267
[1500]	training's rmse: 2.94754	valid_1's rmse: 3.10035
[1600]	training's rmse: 2.93087	valid_1's rmse: 3.09056
[1700]	training's rmse: 2.91622	valid_1's rmse: 3.08174


[I 2023-08-10 08:56:27,138] Trial 98 finished with value: 3.021068020709555 and parameters: {'num_leaves': 136, 'max_depth': 11, 'learning_rate': 0.03687939527050324, 'bagging_fraction': 0.673273301365972, 'feature_fraction': 0.8484093651111115, 'min_split_gain': 0.024861456682896608, 'min_child_samples': 200, 'min_child_weight': 0.029896744322551915, 'lambda_l2': 0.04404233430480502}. Best is trial 73 with value: 2.915392405500049.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback 

Training until validation scores don't improve for 500 rounds
[100]	training's rmse: 3.92603	valid_1's rmse: 3.98244
[200]	training's rmse: 3.5745	valid_1's rmse: 3.63862
[300]	training's rmse: 3.37932	valid_1's rmse: 3.45144
[400]	training's rmse: 3.25374	valid_1's rmse: 3.33539
[500]	training's rmse: 3.17086	valid_1's rmse: 3.26298
[600]	training's rmse: 3.12092	valid_1's rmse: 3.22287
[700]	training's rmse: 3.08202	valid_1's rmse: 3.19283
[800]	training's rmse: 3.0457	valid_1's rmse: 3.16664
[900]	training's rmse: 3.01452	valid_1's rmse: 3.14512
[1000]	training's rmse: 2.98977	valid_1's rmse: 3.12879
[1100]	training's rmse: 2.96927	valid_1's rmse: 3.11638
[1200]	training's rmse: 2.94747	valid_1's rmse: 3.1026
[1300]	training's rmse: 2.92551	valid_1's rmse: 3.08871
[1400]	training's rmse: 2.90377	valid_1's rmse: 3.07549
[1500]	training's rmse: 2.88713	valid_1's rmse: 3.06602
[1600]	training's rmse: 2.87757	valid_1's rmse: 3.06135
[1700]	training's rmse: 2.8772	valid_1's rmse: 3.06104

[I 2023-08-10 08:57:30,645] Trial 99 finished with value: 3.0610396483518136 and parameters: {'num_leaves': 142, 'max_depth': 10, 'learning_rate': 0.0494235286465583, 'bagging_fraction': 0.6513392647156211, 'feature_fraction': 0.7969524066486501, 'min_split_gain': 0.049409630072401796, 'min_child_samples': 191, 'min_child_weight': 0.03841080366250101, 'lambda_l2': 0.054227229816249195}. Best is trial 73 with value: 2.915392405500049.


Number of finished trials:  100
Best trial:
Value:  2.915392405500049
Params: 
    num_leaves: 135
    max_depth: 11
    learning_rate: 0.052586623405764234
    bagging_fraction: 0.6644887700856914
    feature_fraction: 0.8230021324729027
    min_split_gain: 0.0025435434125472204
    min_child_samples: 182
    min_child_weight: 0.023336215175678358
    lambda_l2: 0.0372533252920578


In [ ]:
best_params = study.best_params

best_model = lgb.train(
    best_params,
    train_set=d_train,
    num_boost_round=num_rounds,
    valid_sets=valid_sets,
    verbose_eval=verbose_eval,
    early_stopping_rounds=early_stop,
)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038445 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1861
[LightGBM] [Info] Number of data points in the train set: 318918, number of used features: 30
[LightGBM] [Info] Start training from score 35.693741
Training until validation scores don't improve for 500 rounds
[100]	training's l2: 14.738	valid_1's l2: 15.148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

In [ ]:
pip install joblib

In [ ]:
import joblib

# Setelah melakukan tuning dengan Optuna dan mendapatkan study.best_trial
best_trial = study.best_trial

# Simpan model terbaik menggunakan joblib
best_model_filename = 'best_lgbm_model.joblib'
joblib.dump(best_model, best_model_filename)


['best_lgbm_model.joblib']